In [1]:
!apt-get update 
!apt-get install g++ openjdk-8-jdk python-dev python3-dev 
!pip3 install JPype1-py3 
!pip3 install konlpy 
!JAVA_HOME="C:\Program Files\Java\jdk-16.0.2"

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa

In [2]:
# 나눔 폰트 설치

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 48 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 2s (4,972 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletyp

In [3]:
!apt-get update -qq
!apt-get install fonts-nanum* -qq

Selecting previously unselected package fonts-nanum-eco.
(Reading database ... 149080 files and directories currently installed.)
Preparing to unpack .../fonts-nanum-eco_1.000-6_all.deb ...
Unpacking fonts-nanum-eco (1.000-6) ...
Selecting previously unselected package fonts-nanum-extra.
Preparing to unpack .../fonts-nanum-extra_20170925-1_all.deb ...
Unpacking fonts-nanum-extra (20170925-1) ...
Selecting previously unselected package fonts-nanum-coding.
Preparing to unpack .../fonts-nanum-coding_2.5-1_all.deb ...
Unpacking fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-extra (20170925-1) ...
Setting up fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-eco (1.000-6) ...
Processing triggers for fontconfig (2.12.6-0ubuntu2) ...


In [4]:
# 데이터분석
import pandas as pd
import numpy as np

# 자연어처리 
import konlpy
from nltk import Text
from konlpy.tag import Okt 

# 한글폰트 사용 in colab
%matplotlib inline  
import matplotlib.pyplot as plt
import matplotlib as mpl 
import matplotlib.font_manager as fm  


In [5]:
# 데이터분석
import pandas as pd
import numpy as np

# 자연어처리 
import konlpy
from nltk import Text
from konlpy.tag import Okt 

# 한글폰트 사용 in colab
%matplotlib inline  
import matplotlib.pyplot as plt
import matplotlib as mpl 
import matplotlib.font_manager as fm  


path = '/usr/share/fonts/truetype/nanum/NanumSquareRoundB.ttf'  # 설치된 나눔글꼴중 원하는 녀석의 전체 경로를 가져오자
font_name = fm.FontProperties(fname=path, size=10).get_name()
print(font_name)
plt.rc('font', family=font_name)
mpl.rcParams['axes.unicode_minus'] = False

NanumSquareRound


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
#불러올 파일의 경로를 filename 변수에 저장
filename = '/content/drive/MyDrive/Colab Notebooks/dataset/train.csv'

#pandas read_csv로 불러오기
data = pd.read_csv(filename)
data.head()

,index,제출년도,사업명,사업_부처명,계속과제여부,내역사업명,과제명,요약문_연구목표,요약문_연구내용,요약문_기대효과,요약문_한글키워드,요약문_영문키워드,label
0,0,2016,농업기초기반연구,농촌진흥청,신규,농산물안전성연구,유전정보를 활용한 새로운 해충 분류군 동정기술 개발,○ 새로운 해충분류군의 동정기술 개발 및 유입확산 추적,(가) 외래 및 돌발해충의 발생조사 및 종 동정\n\n\n ○ 대상해충 : 최...,"○ 새로운 돌발 및 외래해충의 신속, 정확한 동정법 향상\n\n\n○ 돌발 및 외래...","뉴클레오티드 염기서열, 분자마커, 종 동정, 침샘, 전사체","nucleotide sequence, molecular marker, species...",24
1,1,2019,이공학학술연구기반구축(R&D),교육부,신규,지역대학우수과학자지원사업(1년~5년),대장암의 TRAIL 내성 표적 인자 발굴 및 TRAIL 반응 예측 유전자 지도 구축...,최종목표: TRAIL 감수성 표적 유전자를 발굴하고 내성제어 기전을 연구. 발굴된...,1차년도\n1) Microarray를 통한 선천적 TRAIL 내성 표적 후보 유전자...,"1) TRAIL 내성 특이적 표적분자를 발굴하고, 이를 이용한 TRAIL 효과 증진...","대장암,항암제 내성,세포사멸,유전자발굴","TRAIL,Colorectal cancer,TRAIL resistance,Apopt...",0
2,2,2016,중소기업기술혁신개발,중소기업청,신규,혁신기업기술개발,비목질계 셀룰로오스 식물자원을 활용한 기능성 부직포 및 고부가가치 뷰티케어 faci...,* 식물계자원 정련 및 최적 신서란 파이버 기초연구 개발\n\n\n* 소비자 및 바...,* 식물계자원 정련 및 최적 신서란 파이버 기초연구 개발\n\n\n- Tencel/...,* 국내 독자적인 비목질계 셀룰로오스 자원의 파이버 및 부직포 제조 등의 기술 확보...,"기능성 샐룰로오스 파이버,천연섬유,기능성 부직포,뷰티&amp;케어 제품,미용 솜","functional cellulose fiber,natural fiber,funct...",0
3,3,2018,창업성장기술개발(R&D),중소벤처기업부,신규,창업사업화연계과제,소화기 암 진단용 분자영상 형광프로브 개발,# 암특이적 바이오마커 발굴 및 바이오마커에 대한 프로브 개발\n\n\n1) 소화...,# 소화기 암 진단용 분자영상 형광프로브 개발\n\n\n- 국소 도포형 소화기 암 ...,# 암 진단기술의 차별성: 소화기 암 특이 프로브 개발\n\n\n- 최근 체외진단시...,"분자 진단,형광 조영제,프로브,항체,대장암","Molecular diagnosis,Fluorescence,probe,antibod...",0
4,4,2016,이공학개인기초연구지원,교육부,계속,기본연구지원,위암환자의 항암제반응예측을 위한 BRCA 발현검사,"수술이 불가능한 위암환자는 생존기간은 10개월 안팎에 지나지 않고, 항암화학요법에 ...",-In situ hybridization 검사의 정확성을 확인하기 위해 위암세포주 ...,-본 연구는 파라핀보관조직에서 in situ hybridization로 mRNA 및...,"BRCA,제자리부합법,조직미세배열,마이크로RNA,위암,항암제반응,젊은 연령/가족성 위암","BRCA,Insituhybridization,tissuemicroarray,micr...",0


In [8]:
#mapping labels
labelsname = '/content/drive/MyDrive/Colab Notebooks/dataset/labels_mapping.csv'

#pandas read_csv로 불러오기
labels = pd.read_csv(labelsname)
labels['소분류'].unique()

array(['00. NaN', '01. 원자력 발전', '02. 핵융합 발전', '03. 청정화력발전·효율화', '04. 수력',
       '05. 태양광', '06. 태양열', '07. 지열', '08. 풍력', '09. 해양에너지',
       '10. 바이오에너지', '11. 폐기물', '12. 수소 제조', '13. 연료전지', '14. 전력 저장',
       '15. 수소 저장', '16. 송배전 시스템', '17. 전기 지능화 기기', '18. 수송 효율화',
       '19. 산업 효율화', '20. 건축 효율화', '21. CCUS', '22. Non-CO2저감 ',
       '23. 유전자원·유전개량', '24. 작물재배·생산', '25. 가축 질병관리', '26. 가공·저장·유통',
       '27. 수계·수생태계', '28. 수자원 확보 및 공급', '29. 수처리', '30. 수재해 관리',
       '31. 기후 예측 및 모델링', '32. 기후 정보 경보 시스템', '33. 해양생태계', '34. 수산자원',
       '35. 연안 재해 관리', '36. 감염 질병 관리', '37. 식품 안전 예방', '38. 산림 생산 증진',
       '39. 산림피해저감', '40. 생태·모니터링·복원', '41. 신재생에너지 하이브리드',
       '42. 저전력 소모 장비', '43. 에너지 하베스팅', '44. 인공광합성', '45. 기타 기후변화 관련 기술'],
      dtype=object)

In [9]:
data['label'].unique()

array([24,  0,  2, 25, 36, 40, 19, 27, 18, 29, 45, 15, 31,  5,  6, 22, 14,
       20, 37, 43,  1, 23, 16,  8, 34, 10, 28, 21, 38, 26, 41, 12,  9, 17,
       33, 13,  3, 39, 32,  4, 30, 11,  7, 35, 44, 42])

## 데이터 탐색
### 판다스 프로파일링

In [10]:
pip install -U pandas-profiling

     |████████████████████████████████| 248 kB 6.4 MB/s 
     |████████████████████████████████| 675 kB 11.9 MB/s 
     |████████████████████████████████| 636 kB 23.9 MB/s 
     |████████████████████████████████| 3.1 MB 35.4 MB/s 
     |████████████████████████████████| 10.1 MB 24.6 MB/s 
     |████████████████████████████████| 102 kB 74.4 MB/s 
     |████████████████████████████████| 62 kB 1.3 MB/s 
     |████████████████████████████████| 812 kB 57.9 MB/s 
     |████████████████████████████████| 28.5 MB 45 kB/s 
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27098 sha256=66e35e2cebc944d67729468979786371499452406951edf292263361a54e919f
  Stored in directory: /root/.cache/pip/wheels/70/e1/52/5b14d250ba868768823940c3229e9950d201a26d0bd3ee8655
  Created wheel for imagehash: filename=ImageHash-4.2.1-py2.py3-none-any.whl size=295207 sha256=a3a3bef2b5f8b2c5189ea73ed81e0a2afd51547aa8e62d70e092b963df42321e
  Stored in directory: /root/.cache/pip/wheels/4c/d5/59/5e3e

In [11]:
# import pandas_profiling

In [12]:
# pr=data.profile_report() # 프로파일링 결과 리포트를 pr에 저장
# pr

In [13]:
# --> 122         category_alias_name = category_alias_name.replace("_", " ")
# AttributeError: 'NoneType' object has no attribute 'replace' 오류로 실행 x, null값 해결할 것 
# pr.to_file('./pr_report.html') # pr_report.html 파일로 저장

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174304 entries, 0 to 174303
Data columns (total 13 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   index      174304 non-null  int64 
 1   제출년도       174304 non-null  int64 
 2   사업명        174304 non-null  object
 3   사업_부처명     174304 non-null  object
 4   계속과제여부     174304 non-null  object
 5   내역사업명      174304 non-null  object
 6   과제명        174304 non-null  object
 7   요약문_연구목표   171302 non-null  object
 8   요약문_연구내용   171303 non-null  object
 9   요약문_기대효과   171253 non-null  object
 10  요약문_한글키워드  171276 non-null  object
 11  요약문_영문키워드  171217 non-null  object
 12  label      174304 non-null  int64 
dtypes: int64(3), object(10)
memory usage: 17.3+ MB


In [15]:
# 라벨순대로 정렬할 것
data.sort_values(by=['label'], axis=0)


,index,제출년도,사업명,사업_부처명,계속과제여부,내역사업명,과제명,요약문_연구목표,요약문_연구내용,요약문_기대효과,요약문_한글키워드,요약문_영문키워드,label
87151,87151,2018,개인기초연구(교육부)(R&D),교육부,계속,기본연구(4년~5년),꿀풀과의 화분 진화 및 약의 발생과정 규명,본 연구에서는 현화식물내 화분의 발달 과정의 특징과 최종 산물인 화분의 형태학적 형...,꽃피는 현화식물의 성공적인 종분화와 진화에 가장 중요한 역할의 중심에는 화분의 진화...,"1) 과내 처음으로 화분발달에 대한 연구를 수행함으로써, 과의 약벽 발생 과정 및 ...","화분,소구상체,계통학 ,형태학 ,진화 ,꿀풀과 ,발생학","pollen ,orbicule,phylogeny ,morphology ,Evolut...",0
110240,110240,2017,산업전문인력역량강화(산업부),산업통상자원부,신규,제조혁신전문인력양성,조선해양산업 퇴직인력 활용 전문인력 양성사업,ㅇ 고숙련 퇴직인력 활용을 통해 생산공정 분야 전문인력 양성\n\n - 전체 사업...,① 조선업 생산공정 전문인력 양성\n\n - (교육과정 개발 및 운영)조선산업 고...,ㅇ 생산공정 현장 전문가의 기술 활용을 통한 전문 인력 양성(연 평균 600명 이상...,"조선해양산업 퇴직인력,산학협력프로젝트,현장 교수,생산공정,재취업 연계","retired workforce,shipbuilding & offshore,reem...",0
110241,110241,2017,개인기초연구(미래부),과학기술정보통신부,계속,자유공모,적층 물질의 분광 특성 제일 원리 연구,"본 연구는, 반금속의 그래핀과 부도체의 육방정 질화붕소 그리고 반도체 특성을 갖는 ...","1. 그래핀, 질화붕소, 전이 금속 칼코겐 화합물등의 이차원 물질들의 분광 특성을 ...",1. 새롭게 구현된 적층 물질의 구조를 통해 특허를 확보. 2. 예측된 적층 물질의...,"제일원리,그래핀,질화붕소,전이금속찰코겐화합물,적층물질,분광학","Firstprinciplecalculation,Graphene,Boronnitrid...",0
110242,110242,2016,농업첨단핵심기술개발사업,농촌진흥청,계속,핵심전략기술개발,맥류 곡실 최적 발효조건 탐색 및 가금 사료화 연구,사료맥류 곡실의 최적 발효사료 생산방법 개발 및 축종별 발효맥류 급여체계 확립,(1) 미생물 및 맥류곡실 종류에 따른 가금사료화 방안 구명(2014년)\n\n\n...,○ 맥류곡실 발효사료의 축산 분야 활용방안 구축 \n\n\n○ 축종별 발효맥류 첨가...,"맥류, 발효, 곡물사료","Barley, fermentation, feed grain",0
110244,110244,2016,신품종지역적응연구,농촌진흥청,계속,신품종개발공동연구,충남지역 과수 우량계통 지역적응시험,"과수 사과, 배, 복숭아, 포도 작목의 충남지역 적응성 검토","사과, 배, 복숭아, 포도 작목의 충남지역 생육, 생태, 과실 특성 검정토","충남지역에 적응성이 뛰언난 사과, 배 작목의 공시 계통 선발토","과수, 신품종, 육성, 이용촉진","fruit tree, new cultivar, breeding, utilizati...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
86370,86370,2017,에너지수요관리핵심기술개발(에특),산업통상자원부,계속,에너지효율향상,난방 온도 2℃이상 향상으로 온실가스 배출량을 10%이상 줄일 수 있고 난방비 절감...,1) 마이크로 미세버블 발생장치 설계 및 개발 (단국대학교와 공동협조)\n\n\n ...,1) 마이크로 미세버블 발생장치 설계 및 개발 (단국대학교와 공동협조)\n\n\n ...,[기술적 측면]\n\n\n o 국내 수도/난방 배관 세척기 개발로 소자본 일자...,"건물 배관 세정 , 건물 난방 , 녹제거 , 열효율 향상 , 마이크로미세버블","Builiding Pipe Cleaning , Building heating , S...",45
86374,86374,2018,기후변화대응기술개발(R&D),과학기술정보통신부,계속,C1가스리파이너리사업,CO 활용 C2-화합물 제조를 위한 유전체 정보기반 Acetogen 기술 개발,CO 활용 C2-화합물 제조를 위한 유전체 정보기반 Acetogen 기술 개발 ...,2단계 (2018-2020년) 일산화탄소 이용 고농도 아세트산 생산관련 기술 (유전...,일산화탄소 발효공정 개량을 통해 일산화탄소 유래 아세트산 생산량의 증대를 기대할 수...,"일산화탄소,계산유전체학,대사네트워크,생물전환,대사회로 재설계","carbon monoxide,computational recombination te...",45
104210,104210,2016,개인연구지원,미래창조과학부,신규,중견연구,실제 대기에서 VOCs의 종류별 SOA marker 분석을 통한 SOA 기여도 산출,실제 대기에서 주요 VOCs 10종의 종류별 Secondary Organic Aer...,1) 자료조사 및 챔버실험 : \n\n- 국내 주요 VOCs 종류 10종 선정: 국...,"- 초미세먼지 내 유기에어로졸의 생성 원인 이해와 연구능력 향상, 그리고 관련 연구...","이차에어로졸, 이차생성 마커, 휘발성 유기오염물질,수용모델, 실제대기,챔버실험,초미세먼지","SecondaryOrganicAerosolmarker,SOAmarker,Volati...",45
132953,132953,2017,국토교통기술촉진연구,국토교통부,계속,창의도전연구,ECG(Eco Clean Cement Grout)기술을 이용한 사면 안정용 항구성 ...,□ 본 연구 개발의 최종 목표는 다음과 같음.\n\n□ ECG 기술을 이용한 사면 ...,□ 본 연구 개발에서는 ECG 기술을 응용한 암반용 식생재료를 개발하여 암반 절개면...,□ 사면의 구조적 안정성 및 내구성 확보\n\n- 비탈면 안정성과 환경복원은 식생이...,"에코 클린 시멘트 그라우트,사면 안정,녹생토,친환경,항구성","Eco Clean Cement Grout,Slope Stability,Green S...",45


In [16]:
# labels가 1인 값 조회, 원자력
num1 = data.loc[data['label'] == 1]
num1

,index,제출년도,사업명,사업_부처명,계속과제여부,내역사업명,과제명,요약문_연구목표,요약문_연구내용,요약문_기대효과,요약문_한글키워드,요약문_영문키워드,label
246,246,2017,원자력연구기반확충사업,과학기술정보통신부,계속,전략기초,고 프란틀 수 용융염의 열전달 현상 이해를 위한 상사유체 기술개발,본 연구 개발의 최종 목표는 고 프란틀 수를 갖는 용융염의 열전달 현상의 이해를 위...,당해연도에서 본 연구 과제를 통하여 연구 개발된 내용은 다음과 같다.\n\n\n\n...,당해연도에서 개발된 연구 내용의 활용 계획 및 기대효과는 다음과 같다\n\n고 프란...,"용융염,전해정련공정,열전달 매체,상사 물질,유체순환,열전달 특성,열수력해석 코드,무...","Moltensalt,Pyroprocessing,Heattransfermedium,S...",1
267,267,2018,원자력기술개발사업(R&D),과학기술정보통신부,계속,원자력원천기술,원자력 발전소 D&D 작업을 위한 광학 힘센싱 로봇 Manipulator 개발,간단하고 견고한 구조에 내장 된 광섬유 센서를 기반으로 하는 힘 감지 기능이 통합된...,구조화되지 않고 예측할 수 없는 D&D 작업에서 복잡하고 능숙한 조작 작업을 수행하...,고방사선 D&D 작업을 효율적으로 수행 할 수 있는 견고한 힘 감지 로봇과 새로운 ...,"원격 제어,힘 제어,힘 감지,광섬유,광섬유 브래그 격자,지능제어","remote control,dexterous manipulation,force se...",1
448,448,2019,한국원자력연구원연구운영비지원(R&D)(주요사업비),과학기술정보통신부,계속,친환경 핵연료주기시스템 실증 및 분석지원,서울 연구로 해체부지 환경복원 및 규제해제,"1. 서울 연구로 1, 2호기 해체부지 환경복원2. 서울 연구로 1, 2호기 해체부...","○ 연구로 1, 2호기 건물 철거1. 서울 연구로 1, 2호기 해체공법 수립 및 해...",1. 우리나라 최초 원자로 해체 및 부지복원(그린필드화) 사업의 모범 사례로 국민에...,"서울 연구용 원자로, 해체, 철거, 해체폐기물, 부지복원","Korea Research Reactor, Decommissioning, Disma...",1
458,458,2018,원자력연구기반확충사업(R&D),과학기술정보통신부,계속,전략기초,봉다발 부수로 내 단상 및 다상유동의 3차원 유동장 측정을 통한 실용적 교차류모델 개발,원자력 발전소의 정상운전 시 성능평가 또는 사고시 안전성진단을 위해서는 부수로 내부...,경수로 부수로 내에서 발생하는 단상 및 다상유동 현상에서 가장 중요한 것은 교차류 ...,고정확도의 봉다발 부수로 내 단상 및 다상유동 측정은 광범위한 활용성을 가지는 실용...,"봉다발 부수로 유동,교차류,고속 2상 입자영상속도계,2상유동,지지격자","Rodbundle subchannel flow,Crossflow,high-speed...",1
684,684,2018,원자력기술개발사업(R&D),과학기술정보통신부,계속,핵연료주기,사고저항성 핵연료 피복관 기술 개발,? 사고저항성 핵연료 피복관 소재 성능/제조 기술 최적화 및 성능평가\n\n (상...,□ 2017\n\n ? 사고저항성 피복관 소재 성능개선\n\n ? 사고저항성 피복관...,? 핵연료 피복관의 고온산화에 따른 수소발생을 감소시킴으로써 중대사고로의 진행을 억...,"핵연료 피복관,표면개질,다층구조,사고저항성,고온산화,고온변형,수소폭발,수소화물,성능평가","Fuel cladding,Surface modification,Multi struc...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173041,173041,2018,원자력핵심기술개발(R&D),산업통상자원부,신규,원자력핵심기술개발(R&D),원전 격납건물 라이너 플레이트 부식손상 관리 기술개발,- 주관기관((주)미래와도전) :\n\n\n \t1) 원전 사이트 비래염분 포집...,- 주관기관((주)미래와도전) :\n\n\n\t1) 원전 사이트 비래염분 포집 시스...,- CLP 구조물은 원전 내부의 방사성물질을 차폐하는 주요 방벽으로 파손시 원전의...,"격납건물 라이너플레이트 부식 , 부식센서 , 콘크리트 전도도 , 부식실증시험 , 부...","Containment Liner plate Corrosion , Corrosion ...",1
173306,173306,2016,원자력기술개발사업,미래창조과학부,계속,소듐냉각고속로개발사업단,SFR원형로 열유체 검증,- IHX 유동특성 시험 데이터베이스 구축\n\n- 원자로 유동분포 시험 데이터베이...,- IHX 유동특성 시험\n\n . IHX 유동특성 시험장치 구축\n\n . I...,- IHX 유동특성 시험 데이터베이스를 전산코드 검증에 활용\n\n- SFR원자로 ...,"소듐냉각 고속로,압력강하,유동분포,열적 여유도,중간 열교환기,검증시험,일체형,입구 ...","SodiumCooledFastReactor,Pressuredrop,Flowdistr...",1
173492,173492,2016,방사선기술개발사업,미래창조과학부,계속,방사능 피해예측 및 저감기술 개발,방사능 식물정화기술 개발,- 방사성핵종 제염효율이 우수한 유용 식물체 확보 및 Lab 규모의 제염효율 평가\...,【15년】 ? 방사성핵종 제염 유용 후보 식물체 탐색 \n\n- 국내외 방사능오염 ...,? 고효율 방사능 제거 후보식물체를 이용한 친환경 방사능 제거 원천핵심기술 확보 ...,"방사성 핵종,제염,식물제염,애기장대,세슘,방사능,제염시스템,방사선,모델링","radionuclides,decontamination,phytoremediation...",1
173692,173692,2018,원자력핵심기술개발(R&D),산업통상자원부,계속,원자력핵심기술개발(R&D),원자력발전소용 3축 편심 양방향 차단 메탈시트 나비형 밸브 개발,- 주관기관((주) 삼신) \n\n\n 1) 2차년도 시작품에 대한 밸브 작동성 ...,- 주관기관((주) 삼신) \n\n\n 1) 2차년도 시작품에 대한 밸브 작동성...,1. 사업화 계획\n\n\n - 현재 웨스팅하우스 AP1000 원전에는 3축 편심 ...,"3축편심 , 메탈시트 , 양방향 차단 , 나비형 밸브 , 원자력 발전소","Triple Offset , Metal Seat , Bidirectional Sea...",1


In [17]:
len(num1)

1261

In [18]:
# 공통적으로 들어가는 단어의 빈도수 확인
# 모든 칼럼 별 확인
# 사업명
# 사업_부처명
# 계속과제여부
# 내역사업명	
# 과제명
# 요약문_연구목표	
# 요약문_연구내용	
# 요약문_기대효과
# 요약문_한글키워드
# 요약문_영문키워드->nltk로 빈도수 분석

In [19]:
from konlpy.tag import Okt 
okt = Okt()

In [20]:
num1['사업명']

246                       원자력연구기반확충사업
267                    원자력기술개발사업(R&D)
448       한국원자력연구원연구운영비지원(R&D)(주요사업비)
458                  원자력연구기반확충사업(R&D)
684                    원자력기술개발사업(R&D)
                     ...             
173041                 원자력핵심기술개발(R&D)
173306                      원자력기술개발사업
173492                      방사선기술개발사업
173692                 원자력핵심기술개발(R&D)
174127                 원자력기술개발사업(R&D)
Name: 사업명, Length: 1261, dtype: object

In [21]:
# 각 칼럼별 텍스트 결합 



In [24]:
num1_buisnessName = []
for i in num1['사업명']:
    num1_buisnessName.append(i)
print(num1_buisnessName)

['원자력연구기반확충사업', '원자력기술개발사업(R&D)', '한국원자력연구원연구운영비지원(R&D)(주요사업비)', '원자력연구기반확충사업(R&D)', '원자력기술개발사업(R&D)', '원자력기술개발사업(R&D)', '원자력핵심기술개발(R&D)', '원자력연구기반확충사업', '한국원자력연구원연구운영비지원', '원자력연구기반확충사업', '원자력기술개발사업(R&D)', 'BK21플러스사업(0.5)', '원자력연구기반확충사업', '원자력기술개발사업(R&D)', '한국과학기술원연구운영비지원(0.5)', '원자력연구기반확충사업(R&D)', '개인기초연구(과기정통부)(R&D)', '원자력기술개발사업(R&D)', '원자력안전연구개발(R&D)', '원자력안전연구개발', '중소기업상용화기술개발(R&D)', '원자력안전연구개발', '원자력핵심기술개발(R&D)', '원자력기술개발사업', '원자력연구기반확충사업', '원자력연구기반확충사업', '원자력연구기반확충사업(R&D)', '원자력기술개발사업', '원자력안전연구개발(R&D)', '원자력핵심기술개발', '원자력핵심기술개발(R&D)', '원자력기술개발사업', '원자력기술개발사업(R&D)', '원자력핵심기술개발', '원자력기술개발사업(R&D)', '원자력핵심기술개발', '원자력안전연구개발', '핵비확산및핵안보이행기술개발(R&D)', '한국기계연구원연구운영비지원(주요사업비)', '원자력기술개발사업', '원자력기술개발사업(R&D)', '원자력연구기반확충사업(R&D)', '원자력기술개발사업', '원자력핵심기술개발', '원자력핵심기술개발(R&D)', '원자력핵심기술개발(R&D)', '원자력핵심기술개발', '원자력안전연구개발', '원자력기술개발사업', '원자력기술개발사업(R&D)', '원자력핵심기술개발(R&D)', '핵비확산및핵안보이행기술개발', '원자력기술개발사업(R&D)', '원자력기술개발사업(R&D)', '원자력핵심기술개발(R&D)', '원자력기술개발사업(R&D)', '원자력기술개발사업(R&D)', '원자력연구기반확충사업', 'SMART

In [26]:
num1_buisnessName_result = ' '.join(num1_buisnessName)
print(num1_buisnessName_result)

원자력연구기반확충사업 원자력기술개발사업(R&D) 한국원자력연구원연구운영비지원(R&D)(주요사업비) 원자력연구기반확충사업(R&D) 원자력기술개발사업(R&D) 원자력기술개발사업(R&D) 원자력핵심기술개발(R&D) 원자력연구기반확충사업 한국원자력연구원연구운영비지원 원자력연구기반확충사업 원자력기술개발사업(R&D) BK21플러스사업(0.5) 원자력연구기반확충사업 원자력기술개발사업(R&D) 한국과학기술원연구운영비지원(0.5) 원자력연구기반확충사업(R&D) 개인기초연구(과기정통부)(R&D) 원자력기술개발사업(R&D) 원자력안전연구개발(R&D) 원자력안전연구개발 중소기업상용화기술개발(R&D) 원자력안전연구개발 원자력핵심기술개발(R&D) 원자력기술개발사업 원자력연구기반확충사업 원자력연구기반확충사업 원자력연구기반확충사업(R&D) 원자력기술개발사업 원자력안전연구개발(R&D) 원자력핵심기술개발 원자력핵심기술개발(R&D) 원자력기술개발사업 원자력기술개발사업(R&D) 원자력핵심기술개발 원자력기술개발사업(R&D) 원자력핵심기술개발 원자력안전연구개발 핵비확산및핵안보이행기술개발(R&D) 한국기계연구원연구운영비지원(주요사업비) 원자력기술개발사업 원자력기술개발사업(R&D) 원자력연구기반확충사업(R&D) 원자력기술개발사업 원자력핵심기술개발 원자력핵심기술개발(R&D) 원자력핵심기술개발(R&D) 원자력핵심기술개발 원자력안전연구개발 원자력기술개발사업 원자력기술개발사업(R&D) 원자력핵심기술개발(R&D) 핵비확산및핵안보이행기술개발 원자력기술개발사업(R&D) 원자력기술개발사업(R&D) 원자력핵심기술개발(R&D) 원자력기술개발사업(R&D) 원자력기술개발사업(R&D) 원자력연구기반확충사업 SMART고도화공동개발(R&D) 국가간협력기반조성(R&D) 원자력기술개발사업 원자력핵심기술개발(R&D) 한국과학기술원연구운영비지원(주요사업비) 원자력기술개발사업(R&D) 원자력기술개발사업(R&D) 방사선기술개발사업(R&D) 원자력기술개발사업(R&D) 원자력기술개발사업 원자력안전연구개발(R&D) 원자력연구기반확충사업 달탐사 

In [32]:
type(num1_buisnessName_result)

str

In [36]:
# 명사 추출
num1_buisnessName_result_noun = okt.nouns(num1_buisnessName_result)
num1_buisnessName_result_noun

['원자력',
 '확충',
 '사업',
 '원자력',
 '기술',
 '개발',
 '사업',
 '한국',
 '원자력',
 '연구원',
 '연구',
 '운영',
 '비지',
 '사업',
 '비',
 '원자력',
 '확충',
 '사업',
 '원자력',
 '기술',
 '개발',
 '사업',
 '원자력',
 '기술',
 '개발',
 '사업',
 '원자력',
 '핵심',
 '기술',
 '개발',
 '원자력',
 '확충',
 '사업',
 '한국',
 '원자력',
 '연구원',
 '연구',
 '운영',
 '비지',
 '원자력',
 '확충',
 '사업',
 '원자력',
 '기술',
 '개발',
 '사업',
 '플러스',
 '사업',
 '원자력',
 '확충',
 '사업',
 '원자력',
 '기술',
 '개발',
 '사업',
 '한국',
 '과학기술원',
 '운영',
 '비지',
 '원자력',
 '확충',
 '사업',
 '개인',
 '기초',
 '연구',
 '과',
 '정통부',
 '원자력',
 '기술',
 '개발',
 '사업',
 '원자력',
 '안전',
 '연구개발',
 '원자력',
 '안전',
 '연구개발',
 '중소기업',
 '상용',
 '기술',
 '개발',
 '원자력',
 '안전',
 '연구개발',
 '원자력',
 '핵심',
 '기술',
 '개발',
 '원자력',
 '기술',
 '개발',
 '사업',
 '원자력',
 '확충',
 '사업',
 '원자력',
 '확충',
 '사업',
 '원자력',
 '확충',
 '사업',
 '원자력',
 '기술',
 '개발',
 '사업',
 '원자력',
 '안전',
 '연구개발',
 '원자력',
 '핵심',
 '기술',
 '개발',
 '원자력',
 '핵심',
 '기술',
 '개발',
 '원자력',
 '기술',
 '개발',
 '사업',
 '원자력',
 '기술',
 '개발',
 '사업',
 '원자력',
 '핵심',
 '기술',
 '개발',
 '원자력',
 '기술',
 '개발',
 '사업',
 '원자력',
 '핵심',
 '기술',
 '개발',
 

In [37]:
# ValueError: Iterable over raw text documents expected, string object received. 
# 오류 날 시 string에 list 표시하여 바꾸기

# CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer()
vect.fit(num1_buisnessName_result_noun)
vect.vocabulary_

{'강화': 0,
 '개발': 1,
 '개인': 2,
 '거점': 3,
 '검증': 4,
 '고도화': 5,
 '공기업': 6,
 '공동': 7,
 '공학': 8,
 '과학기술원': 9,
 '관리': 10,
 '교류': 11,
 '교육부': 12,
 '구지원': 13,
 '구축': 14,
 '국가': 15,
 '국기': 16,
 '국제': 17,
 '규제': 18,
 '기관지': 19,
 '기금': 20,
 '기반': 21,
 '기술': 22,
 '기초': 23,
 '기획': 24,
 '미래부': 25,
 '및핵': 26,
 '방사선': 27,
 '방폐물': 28,
 '보급': 29,
 '복합': 30,
 '부품': 31,
 '비지': 32,
 '사업': 33,
 '산업': 34,
 '산학': 35,
 '상용': 36,
 '소재': 37,
 '수출': 38,
 '순환': 39,
 '시스템': 40,
 '신진': 41,
 '실증': 42,
 '안보': 43,
 '안전': 44,
 '에너지': 45,
 '에너지국': 46,
 '역량': 47,
 '연구': 48,
 '연구개발': 49,
 '연구원': 50,
 '연구자': 51,
 '요소': 52,
 '용신': 53,
 '우수': 54,
 '우주': 55,
 '운영': 56,
 '원자력': 57,
 '유치': 58,
 '융합': 59,
 '이행': 60,
 '인기': 61,
 '인력': 62,
 '인증': 63,
 '자원': 64,
 '재료연': 65,
 '전력': 66,
 '전문': 67,
 '정부': 68,
 '정통부': 69,
 '조성': 70,
 '중소기업': 71,
 '지원': 72,
 '초연': 73,
 '탐사': 74,
 '통제': 75,
 '평가': 76,
 '표준': 77,
 '표준화': 78,
 '플러스': 79,
 '학술': 80,
 '한국': 81,
 '해외': 82,
 '해체': 83,
 '핵비': 84,
 '핵심': 85,
 '혁신': 86,
 '협력': 87,
 '확산': 88,
 '확충'

In [ ]:
# 자연어처리 목적 : 공통적으로 원자력과 관련 있는 단어 추출
# 해당 단어가 포함된 값은 label을 '01. 원자력 발전'라고 함, 02. 핵융합발전과 겹칠 수 있으니 어떻게 구별할지 유의
# bow인코딩벡터로 변환하여 해당 값이 있는 칼럼만 선정해서 훈련??
# 토큰이 나타난 횟수를 기준으로 단어장 구성 가능 

# '및핵': 26,
# '방사선': 27,
# '방폐물': 28,
# '원자력': 57,
# '융합': 59,
# '핵비': 84,

In [38]:
# num1사업명 칼럼을 Bow인코딩 벡터로 변환
# 앞의 토큰 리스트 순서대로 없는 토큰은 0, 있는 토큰은 1로 나타냄 
vect.transform(num1['사업명']).toarray()

# 타 칼럼도 똑같이 분석 후 전체 데이터셋에 적용 
# 관련 있는 단어면 칼럼이 달라도 똑같이 적용하면 될 지??

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
# 요약문 영문 키워드는 nltk 라이브러리로 분석

In [ ]:
pip install "nltk==3.4.5"

In [ ]:
import nltk

In [ ]:
# 단어단위 토큰 문자열 리스트 출력
from nltk.tokenizer import word_tokenize
sentence = "data['요약문_영문키워드']"
word_tokenize(sentence)